In [1]:
!cp -r ../input/recsys-repo/RecSys_Course_AT_PoliMi-master/* ./
%config Completer.use_jedi = False
%load_ext Cython
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

In [2]:
!python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969,
                 from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:746:
/opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorizationImpressions_Cython_Epoch.c: In function ‘__

In [3]:
ICM_type_df = pd.read_csv("/kaggle/input/competition-data/data_ICM_type.csv")
ICM_type_df

items = ICM_type_df.item_id
features = ICM_type_df.feature_id
data = ICM_type_df.data
ICM_type = sps.csr_matrix((data, (items, features)))
ICM_type = ICM_type.astype(dtype = np.int32)

n_users = 41629
n_itemsFromICM = ICM_type.shape[0]
ICM_type.shape

(27968, 8)

In [4]:
URM_train = sps.load_npz("/kaggle/input/reeborooba/trainDefault.npz")
URM_train = sps.csr_matrix((URM_train.data, URM_train.indices, URM_train.indptr), shape=(n_users, n_itemsFromICM))
URM_train

<41629x27968 sparse matrix of type '<class 'numpy.float64'>'
	with 1321444 stored elements in Compressed Sparse Row format>

In [5]:
URM_valid = sps.load_npz("/kaggle/input/reeborooba/validDefault.npz")
URM_valid = sps.csr_matrix((URM_valid.data, URM_valid.indices, URM_valid.indptr), shape=(n_users, n_itemsFromICM))
URM_valid

<41629x27968 sparse matrix of type '<class 'numpy.float64'>'
	with 233196 stored elements in Compressed Sparse Row format>

In [6]:
URM_train_stacked = sps.vstack([URM_train, ICM_type.T])
URM_train_stacked = sps.csr_matrix(URM_train_stacked)

In [7]:
#create an evaluator object to evaluate validation set
#we will use it for hyperparameter tuning
evaluator_valid = EvaluatorHoldout(URM_valid, cutoff_list=[10])

EvaluatorHoldout: Ignoring 963 ( 2.3%) Users that have less than 1 test interactions


In [8]:
import implicit
from Recommenders.BaseMatrixFactorizationRecommender import BaseMatrixFactorizationRecommender
import scipy.sparse as sps
class ImplicitALSRecommender(BaseMatrixFactorizationRecommender):
    """ImplicitALSRecommender recommender"""

    RECOMMENDER_NAME = "ImplicitALSRecommender"

    def fit(self,
            factors=100,
            regularization=0.01,
            use_native=True, use_cg=True, use_gpu=False,
            iterations=15,
            calculate_training_loss=False, num_threads=0,
            confidence_scaling=None,
            icm_coeff = 1,
            **confidence_args
            ):
        self.rec = implicit.als.AlternatingLeastSquares(factors=factors, regularization=regularization,
                                                        use_native=use_native, use_cg=use_cg, use_gpu=use_gpu,
                                                        iterations=iterations,
                                                        calculate_training_loss=calculate_training_loss,
                                                        num_threads=num_threads,
                                                        random_state=5)
        self.rec.fit(confidence_scaling(self.URM_train, **confidence_args), show_progress=self.verbose)

        self.USER_factors = self.rec.user_factors
        self.ITEM_factors = self.rec.item_factors

In [9]:
import numpy as np
import scipy.sparse as sps
from Recommenders.Recommender_utils import check_matrix
from sklearn.linear_model import ElasticNet
from Recommenders.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit
import time, sys
from tqdm import tqdm
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

# os.environ["PYTHONWARNINGS"] = ('ignore::exceptions.ConvergenceWarning:sklearn.linear_model')
# os.environ["PYTHONWARNINGS"] = ('ignore:Objective did not converge:ConvergenceWarning:')

class SLIMElasticNetRecommender(BaseItemSimilarityMatrixRecommender):
    """
    Train a Sparse Linear Methods (SLIM) item similarity model.
    NOTE: ElasticNet solver is parallel, a single intance of SLIM_ElasticNet will
          make use of half the cores available
    See:
        Efficient Top-N Recommendation by Linear Regression,
        M. Levy and K. Jack, LSRS workshop at RecSys 2013.
        SLIM: Sparse linear methods for top-n recommender systems,
        X. Ning and G. Karypis, ICDM 2011.
        http://glaros.dtc.umn.edu/gkhome/fetch/papers/SLIM2011icdm.pdf
    """

    RECOMMENDER_NAME = "SLIMElasticNetRecommender"

    def __init__(self, URM_train, verbose = True):
        super(SLIMElasticNetRecommender, self).__init__(URM_train, verbose = verbose)

    @ignore_warnings(category=ConvergenceWarning)
    def fit(self, l1_ratio=0.1, alpha = 1.0, positive_only=True, topK = 100,**earlystopping_kwargs):

        assert l1_ratio>= 0 and l1_ratio<=1, "{}: l1_ratio must be between 0 and 1, provided value was {}".format(self.RECOMMENDER_NAME, l1_ratio)

        self.l1_ratio = l1_ratio
        self.positive_only = positive_only
        self.topK = topK


        # initialize the ElasticNet model
        self.model = ElasticNet(alpha=alpha,
                                l1_ratio=self.l1_ratio,
                                positive=self.positive_only,
                                fit_intercept=False,
                                copy_X=False,
                                precompute=True,
                                selection='random',
                                max_iter=100,
                                tol=1e-4)

        URM_train = check_matrix(self.URM_train, 'csc', dtype=np.float32)

        n_items = URM_train.shape[1]

        # Use array as it reduces memory requirements compared to lists
        dataBlock = 10000000

        rows = np.zeros(dataBlock, dtype=np.int32)
        cols = np.zeros(dataBlock, dtype=np.int32)
        values = np.zeros(dataBlock, dtype=np.float32)

        numCells = 0

        start_time = time.time()
        start_time_printBatch = start_time

        # fit each item's factors sequentially (not in parallel)
        for currentItem in range(n_items):

            # get the target column
            y = URM_train[:, currentItem].toarray()

            # set the j-th column of X to zero
            start_pos = URM_train.indptr[currentItem]
            end_pos = URM_train.indptr[currentItem + 1]

            current_item_data_backup = URM_train.data[start_pos: end_pos].copy()
            URM_train.data[start_pos: end_pos] = 0.0

            # fit one ElasticNet model per column
            self.model.fit(URM_train, y)

            # self.model.coef_ contains the coefficient of the ElasticNet model
            # let's keep only the non-zero values

            # Select topK values
            # Sorting is done in three steps. Faster then plain np.argsort for higher number of items
            # - Partition the data to extract the set of relevant items
            # - Sort only the relevant items
            # - Get the original item index

            nonzero_model_coef_index = self.model.sparse_coef_.indices
            nonzero_model_coef_value = self.model.sparse_coef_.data

            local_topK = min(len(nonzero_model_coef_value)-1, self.topK)

            relevant_items_partition = (-nonzero_model_coef_value).argpartition(local_topK)[0:local_topK]
            relevant_items_partition_sorting = np.argsort(-nonzero_model_coef_value[relevant_items_partition])
            ranking = relevant_items_partition[relevant_items_partition_sorting]

            for index in range(len(ranking)):

                if numCells == len(rows):
                    rows = np.concatenate((rows, np.zeros(dataBlock, dtype=np.int32)))
                    cols = np.concatenate((cols, np.zeros(dataBlock, dtype=np.int32)))
                    values = np.concatenate((values, np.zeros(dataBlock, dtype=np.float32)))


                rows[numCells] = nonzero_model_coef_index[ranking[index]]
                cols[numCells] = currentItem
                values[numCells] = nonzero_model_coef_value[ranking[index]]

                numCells += 1

            # finally, replace the original values of the j-th column
            URM_train.data[start_pos:end_pos] = current_item_data_backup

            elapsed_time = time.time() - start_time
            new_time_value, new_time_unit = seconds_to_biggest_unit(elapsed_time)


            if time.time() - start_time_printBatch > 300 or currentItem == n_items-1:
                self._print("Processed {} ({:4.1f}%) in {:.2f} {}. Items per second: {:.2f}".format(
                    currentItem+1,
                    100.0* float(currentItem+1)/n_items,
                    new_time_value,
                    new_time_unit,
                    float(currentItem)/elapsed_time))

                sys.stdout.flush()
                sys.stderr.flush()

                start_time_printBatch = time.time()

        # generate the sparse weight matrix
        self.W_sparse = sps.csr_matrix((values[:numCells], (rows[:numCells], cols[:numCells])),
                                       shape=(n_items, n_items), dtype=np.float32)

In [10]:
from Recommenders.BaseMatrixFactorizationRecommender import BaseMatrixFactorizationRecommender
from Recommenders.Incremental_Training_Early_Stopping import Incremental_Training_Early_Stopping
from Recommenders.Recommender_utils import check_matrix

def linear_scaling_confidence(URM_train, alpha):
    C = check_matrix(URM_train.T, format="csr", dtype=np.float32)
    C.data = 1.0 + alpha * C.data

    return C

In [11]:
import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

In [12]:
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython

slim_bpr = SLIM_BPR_Cython(URM_train)
slim_bpr.fit(epochs=650, sgd_mode = "sgd", topK = 483, lambda_i = 0.0006712905081189398, lambda_j = 0.06584150350451998, learning_rate = 0.0036482363905043207)

SLIM_BPR_Recommender: URM Detected 3461 (12.4%) items with no interactions.
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Available RAM is 30777.00 MB (95.85%) of 32110.00 MB, required is 3128.84 MB. Using dense matrix.
Processed 41629 (100.0%) in 1.13 sec. BPR loss is 2.69E-03. Sample per second: 36708
SLIM_BPR_Recommender: Epoch 1 of 650. Elapsed time 0.26 sec
Processed 41629 (100.0%) in 0.39 sec. BPR loss is 1.71E-02. Sample per second: 106222
SLIM_BPR_Recommender: Epoch 2 of 650. Elapsed time 0.51 sec
Processed 41629 (100.0%) in 0.65 sec. BPR loss is 3.71E-02. Sample per second: 64378
SLIM_BPR_Recommender: Epoch 3 of 650. Elapsed time 0.77 sec
Processed 41629 (100.0%) in 0.90 sec. BPR loss is 6.29E-02. Sample per second: 46384
SLIM_BPR_Recommender: Epoch 4 of 650. Elapsed time 1.02 sec
Processed 41629 (100.0%) in 1.15 sec. BPR loss is 9.29E-02. Sample per second: 36217
SLIM_BPR_Recommender: Epoch 5 of 650. Elapsed time 1.27 sec
Processed 41629 (100.0%) in 0.41 se

In [13]:
recommender_SLIMElasticNet1 = SLIMElasticNetRecommender(URM_train_stacked)
recommender_SLIMElasticNet1.fit(l1_ratio= 0.02663352009457766, alpha= 0.0010777130010163724, positive_only= True, topK= 15368)

SLIMElasticNetRecommender: URM Detected 3 ( 0.0%) users with no interactions.
SLIMElasticNetRecommender: Processed 2774 ( 9.9%) in 5.00 min. Items per second: 9.24
SLIMElasticNetRecommender: Processed 5666 (20.3%) in 10.00 min. Items per second: 9.44
SLIMElasticNetRecommender: Processed 8792 (31.4%) in 15.00 min. Items per second: 9.77
SLIMElasticNetRecommender: Processed 11864 (42.4%) in 20.00 min. Items per second: 9.88
SLIMElasticNetRecommender: Processed 14937 (53.4%) in 25.00 min. Items per second: 9.96
SLIMElasticNetRecommender: Processed 18162 (64.9%) in 30.00 min. Items per second: 10.09
SLIMElasticNetRecommender: Processed 21342 (76.3%) in 35.01 min. Items per second: 10.16
SLIMElasticNetRecommender: Processed 24482 (87.5%) in 40.01 min. Items per second: 10.20
SLIMElasticNetRecommender: Processed 27968 (100.0%) in 41.12 min. Items per second: 11.34


In [14]:
RP3beta_all = RP3betaRecommender(URM_train)
RP3beta_all.fit(alpha= 0.8048209317587374, beta= 0.28523965008406016, topK= 56, implicit= False)

RP3betaRecommender: URM Detected 3461 (12.4%) items with no interactions.
RP3betaRecommender: Similarity column 27968 (100.0%), 2408.88 column/sec. Elapsed time 11.61 sec


In [16]:
slim_matrix1 = recommender_SLIMElasticNet1.W_sparse
rp3_matrix = RP3beta_all.W_sparse
bpr_matrix = slim_bpr.W_sparse

In [17]:
result_dict, _ = evaluator_valid.evaluateRecommender(recommender_SLIMElasticNet1)
print(f"Slim_en: {result_dict['MAP'][10]}")

result_dict, _ = evaluator_valid.evaluateRecommender(RP3beta_all)
print(f"Rp3: {result_dict['MAP'][10]}")

EvaluatorHoldout: Processed 40666 (100.0%) in 52.12 sec. Users per second: 780
Slim_en: 0.02783886484477793
EvaluatorHoldout: Processed 40666 (100.0%) in 32.75 sec. Users per second: 1242
Rp3: 0.025709229034979467


In [18]:
from Recommenders.Recommender_utils import check_matrix, similarityMatrixTopK
from Recommenders.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender



class ItemKNNSimilarityHybridRecommender(BaseItemSimilarityMatrixRecommender):
    """ ItemKNNSimilarityHybridRecommender
    Hybrid of two similarities S = S1*alpha + S2*(1-alpha)
    """

    RECOMMENDER_NAME = "ItemKNNSimilarityHybridRecommender"


    def __init__(self, URM_train, Similarity_1, Similarity_2, verbose = True):
        super(ItemKNNSimilarityHybridRecommender, self).__init__(URM_train, verbose = verbose)

        # CSR is faster during evaluation
        self.Similarity_1 = check_matrix(Similarity_1.copy(), 'csr')
        self.Similarity_2 = check_matrix(Similarity_2.copy(), 'csr')


    def fit(self, topK=100, alpha = 0.5, beta = 0):

        self.topK = topK
        self.alpha = alpha
        self.beta = beta


        W_sparse = self.Similarity_1*self.alpha + self.Similarity_2*self.beta

        self.W_sparse = similarityMatrixTopK(W_sparse, k=self.topK)
        self.W_sparse = check_matrix(self.W_sparse, format='csr')

In [ ]:
from numpy import linalg as LA
from Recommenders.BaseRecommender import BaseRecommender

class DifferentLossScoresHybridRecommender(BaseRecommender):

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2, recommender_3, recommender_4):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        self.recommender_3 = recommender_3
        self.recommender_4 = recommender_4
        
        
        
    def fit(self, alpha = 0.5, beta = 0, gamma = 0, theta = 0):

        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.theta = theta



    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)
        item_weights_3 = self.recommender_3._compute_item_score(user_id_array)
        item_weights_4 = self.recommender_4._compute_item_score(user_id_array)
            
        item_weights = item_weights_1 * self.alpha + item_weights_2 * self.beta + item_weights_3 * self.gamma + item_weights_4 * self.theta

        return item_weights

In [21]:
recommender_class = ItemKNNSimilarityHybridRecommender

import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 1000
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [22]:
from skopt.space import Real, Integer, Categorical
#RP3 beta
#to tuning hyperparam are typical of ML models to drive the learning process

hyperparameters_range_dictionary = {
    "topK": Integer(100,2000),
    "alpha": Real(low=0, high=100, prior='uniform'),
    "beta": Real(low=0, high=50, prior='uniform')
}

from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

#create a bayesian optimizer object, we pass the recommender and the evaluator
hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_valid)
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
#provide data needed to create instance of model (one on URM_train, the other on URM_all)
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train, rp3_matrix, bpr_matrix],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [ ]:
hyperparameterSearch.search(recommender_input_args = recommender_input_args,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )

SearchBayesianSkopt: argument save_model is 'last' but no recommender_input_args_last_test provided, saving best model on train data alone.
Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 1520, 'alpha': 99.96013008102457, 'beta': 24.71303956238881}
ItemKNNSimilarityHybridRecommender: URM Detected 3461 (12.4%) items with no interactions.
EvaluatorHoldout: Processed 40666 (100.0%) in 48.52 sec. Users per second: 838
SearchBayesianSkopt: New best config found. Config 0: {'topK': 1520, 'alpha': 99.96013008102457, 'beta': 24.71303956238881} - results: PRECISION: 0.0449688, PRECISION_RECALL_MIN_DEN: 0.0778021, RECALL: 0.0734357, MAP: 0.0204603, MAP_MIN_DEN: 0.0349393, MRR: 0.1481533, NDCG: 0.0721226, F1: 0.0557802, HIT_RATE: 0.3230217, ARHR_ALL_HITS: 0.1732853, NOVELTY: 0.0034418, AVERAGE_POPULARITY: 0.3222603, DIVERSITY_MEAN_INTER_LIST: 0.8034911, DIVERSITY_HERFINDAHL: 0.9803471, COVERAGE_ITEM: 0.2644451, COVERAGE_ITEM_HIT: 0.02899

In [ ]:
from skopt.space import Real, Integer, Categorical
#RP3 beta
#to tuning hyperparam are typical of ML models to drive the learning process

hyperparameters_range_dictionary = {
    "alpha": Real(low=99.17866170336520, high=99.17866170336522, prior='uniform'),
    "beta": Real(low=40.45134622013221, high=40.45134622013223, prior='uniform'),
    "gamma": Real(low=10.962721766445173, high=10.962721766445175, prior='uniform'),
    "theta": Real(low=23.275965228118105, high=23.275965228118107, prior='uniform'),
    "sigma": Real(low=0, high=20, prior='uniform'),
    "epsilon": Real(low=0, high=60, prior='uniform')

}

from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

#create a bayesian optimizer object, we pass the recommender and the evaluator
hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_valid)
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
#provide data needed to create instance of model (one on URM_train, the other on URM_all)
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train, recommender_SLIMElasticNet1, RP3beta_all, EASE_R, recommender_implicitAls, itemknn_cf, slim_bpr],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [ ]:
#let's run the bayesian search
hyperparameterSearch.search(recommender_input_args,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )